# Historical tweet counts, by term

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt
import altair_stiles as altstiles

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

---

## Search for a term

In [5]:
term = "cities burned blm"

#### Get daily counts

In [7]:
!twarc2 counts --granularity 'day' '{term}' --archive --csv data/raw/burn.csv

  0%|                   | Processed a moment/16 years [00:00<?, 0 tweets total ]
⚡ Client Forbidden


In [7]:
src = pd.read_csv(f"data/raw/{term}.csv")

In [8]:
src

,start,end,day_count
0,2022-06-03T00:00:00.000Z,2022-06-04T00:00:00.000Z,185
1,2022-06-04T00:00:00.000Z,2022-06-05T00:00:00.000Z,88
2,2022-06-05T00:00:00.000Z,2022-06-06T00:00:00.000Z,92
3,2022-06-06T00:00:00.000Z,2022-06-07T00:00:00.000Z,33
4,2022-06-07T00:00:00.000Z,2022-06-08T00:00:00.000Z,33
...,...,...,...
5944,2006-04-13T00:00:00.000Z,2006-04-14T00:00:00.000Z,0
5945,2006-04-14T00:00:00.000Z,2006-04-15T00:00:00.000Z,0
5946,2006-04-15T00:00:00.000Z,2006-04-16T00:00:00.000Z,0
5947,2006-04-16T00:00:00.000Z,2006-04-17T00:00:00.000Z,0


In [9]:
src["year"] = pd.to_datetime(src["start"]).dt.strftime("%Y")
src["month_year"] = pd.to_datetime(src["start"]).dt.strftime("%Y-%m")
src["date"] = pd.to_datetime(src["start"]).dt.strftime("%Y-%m-%d")

In [10]:
src[src["date"] == src["date"].max()]

,start,end,day_count,year,month_year,date
30,2022-07-03T00:00:00.000Z,2022-07-03T14:51:38.000Z,7455,2022,2022-07,2022-07-03


In [11]:
src_grouped = (
    src.groupby(["date", "year", "month_year"]).agg({"day_count": sum}).reset_index()
)

#### Slim down and re-order the dataframe

In [12]:
df = (
    src_grouped[["year", "month_year", "date", "day_count"]]
    .sort_values("date", ascending=False)
    .copy()
)

In [13]:
df.head()

,year,month_year,date,day_count
5948,2022,2022-07,2022-07-03,7455
5947,2022,2022-07,2022-07-02,15508
5946,2022,2022-07,2022-07-01,16529
5945,2022,2022-06,2022-06-30,49749
5944,2022,2022-06,2022-06-29,23849


#### When was the first mention? 

In [14]:
first = df[df["day_count"] > 0]["date"].tail(1).iloc[0]
first

'2007-03-19'

#### How many total mentions? 

In [15]:
df.day_count.sum()

896284

#### Average mentions? 

In [16]:
df.day_count.mean()

150.66128761136326

#### Make a new dataframe starting from first mention

In [17]:
df_complete = df[df["date"] >= first]

#### Which day was mentioned the most? 

In [18]:
df_complete[df_complete["day_count"] == df_complete["day_count"].max()]

,year,month_year,date,day_count
2005,2011,2011-09,2011-09-16,63895


In [19]:
df_complete.sort_values("day_count", ascending=False).head(10)

,year,month_year,date,day_count
2005,2011,2011-09,2011-09-16,63895
5945,2022,2022-06,2022-06-30,49749
2876,2014,2014-02,2014-02-03,34459
5944,2022,2022-06,2022-06-29,23849
5946,2022,2022-07,2022-07-01,16529
5947,2022,2022-07,2022-07-02,15508
5884,2022,2022-04,2022-04-30,11500
2442,2012,2012-11,2012-11-26,8436
5933,2022,2022-06,2022-06-18,7468
5948,2022,2022-07,2022-07-03,7455


#### Chart it

In [20]:
alt.Chart(df_complete[df_complete["date"] > "2022-01-01"]).mark_bar().encode(
    x=alt.X("date:T", axis=alt.Axis(format="%b. %Y", tickCount=5), title=""),
    y=alt.Y("day_count:Q", title="", axis=alt.Axis(tickCount=5, format="0,M")),
).properties(width=650, height=300, title=f"Daily mentions of {term} on Twitter")

alt.Chart(...)

---

## Aggregate 

#### Groupby month/year

In [21]:
months = (
    df_complete.groupby(["month_year"])
    .agg({"day_count": sum})
    .sort_values("month_year", ascending=False)
    .reset_index()
)

#### Chart it

In [22]:
alt.Chart(months.head(120)).mark_area().encode(
    x=alt.X("month_year:T", axis=alt.Axis(format="%b. %Y", tickCount=5), title=""),
    y=alt.Y("day_count:Q", title=" ", axis=alt.Axis(tickCount=5, format="0,M")),
).properties(width=650, title=f"Monthly mentions of {term} on Twitter")

alt.Chart(...)

## Exports

In [23]:
months[months["month_year"] > "2014-12"].to_csv(
    f"data/processed/twitter_mentions_{term}_months.csv", index=False
)
df_complete[df_complete["date"] > "2014-12-31"].to_csv(
    f"data/processed/twitter_mentions_{term}_days.csv", index=False
)

In [24]:
months.head()

,month_year,day_count
0,2022-07,39492
1,2022-06,112133
2,2022-05,57073
3,2022-04,18919
4,2022-03,797


In [25]:
months[months["month_year"] > "2021-01"].day_count.mean()

13350.388888888889